<a href="https://colab.research.google.com/github/fzanart/Socialz/blob/main/data_characterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')
import json
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Socialz/cobol_pullrequests.csv')
df

In [ ]:
users = df['login'].unique()

In [ ]:
def text_lenght(text):

    if text == None:

        return 0
    else:

        return len(text)

df['payload'] = df['payload'].apply(json.loads)
df['merged'] = df['payload'].apply(lambda x: x.get('pull_request').get('merged'))
df['comments'] = df['payload'].apply(lambda x: x.get('pull_request').get('comments'))
df['review_comments'] = df['payload'].apply(lambda x: x.get('pull_request').get('review_comments'))
df['title lenght'] = df['payload'].apply(lambda x: text_lenght(x.get('pull_request').get('title')))
df['body lenght'] = df['payload'].apply(lambda x: text_lenght(x.get('pull_request').get('body')))

In [ ]:
columns = ['merged', 'commits', 'additions', 'deletions','changed_files', 'comments', 'review_comments','title lenght', 'body lenght']

def outlier_detection(df, user):
  
  # get train / test data by user
  X_train = df[(df['merged'] == True) & (df['login'] == user)]
  X_train = X_train[columns]

  X_test = df[(df['merged'] == False) & (df['login'] == user)]
  X_test = X_test[columns]

  # fit predict isolation forest algorithm for outlier detection
  clf = IsolationForest(max_samples='auto', n_jobs=-1, random_state=42)
  clf.fit(X_train)
  y_pred_train = clf.fit_predict(X_train)
  y_pred_test = clf.predict(X_test)

  # Concatenate train / test data with outlier predictions
  anomaly = np.concatenate([y_pred_train,y_pred_test])
  data = pd.concat([X_train, X_test])
  data['Anomaly'] = anomaly

  return data['Anomaly']

In [ ]:
anomaly = pd.Series()

for user in users:
  anomaly = pd.concat([anomaly, outlier_detection(df, user)])

In [ ]:
#TODO: add abnormal ratio, add average monthly activity -> add date/time data, user type, calculate centrality degree

In [ ]:
# Build graph to calculate degree of centrality:

df['login'] = df['login'].apply(lambda x: 'User: '+x)
df['name'] = df['name'].apply(lambda x: 'Repo: '+x)

# Build the adjacency matrix
am = pd.crosstab(df['login'], df['name']).astype(float)
idx = am.columns.union(am.index)
am = am.reindex(index = idx, columns=idx, fill_value=0.0)

matrix_elements = np.sort(df['login'].unique(), axis=0)
repo_len = len(df['name'].unique())

def  cosine_similarity(vector_a, vector_b):
    
    norma = (np.linalg.norm(vector_a)*np.linalg.norm(vector_b))

    if norma > 0:

        cos_sim = np.dot(vector_a,vector_b)/ norma

    else:

        cos_sim = 0

    return cos_sim

#Initialise an auxiliar matrix to record the user - user similarities
shape = (len(matrix_elements), len(matrix_elements))
aux_matrix = pd.DataFrame((np.zeros(shape=shape)), columns=matrix_elements, index=matrix_elements)

for i in matrix_elements:
    for j in matrix_elements:

        # Compare row vectors
        aux_vector_a = am.loc[i].to_numpy()[:repo_len]
        aux_vector_b = am.loc[j].to_numpy()[:repo_len]
        
        score = cosine_similarity(aux_vector_a, aux_vector_b)
        # print(i, j, score)
        aux_matrix.at[i, j] = score

#%%
# Fill adj_matrix with aux_matrix data
for column in aux_matrix.columns:
    for row in aux_matrix.index:

        aux = aux_matrix.at[row, column]
        am.at[row, column] = aux
        # if (aux == am.at[row, column]) ==  False:
        #     print(aux, row, column)

# %%
# Build graph from matrix, after user-user connections
G = nx.from_numpy_matrix(am.to_numpy())

In [ ]:
def CreateBarChartForCentralityMeasures(data, centrality_name):
    df = pd.DataFrame(data, columns=['Node', centrality_name])
    return df

degree_centrality_df = CreateBarChartForCentralityMeasures(sorted(nx.degree_centrality(G).items(), key=lambda x:x[1], reverse=True), 'Degree Centrality')

In [ ]:
# Calculate average monthly activity:


def timelist_to_minutes(time_list):

    datetime_list = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S %Z') for x in time_list]

    minutes = [time.hour*60 + time.minute + time.second/60 for time in datetime_list]

    return minutes

def time_to_minutes(time):

    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S %Z')

    minutes = time.hour*60 + time.minute + time.second/60

    return minutes


df['Time_in_minutes'] = df['created_at'].apply(lambda x: time_to_minutes(x))